<a href="https://colab.research.google.com/github/UU-IM-EU/DEMO-ML-AI/blob/main/BERTopic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>




## BERTopic
BERTopic is a topic modeling technique that leverages 🤗 transformers and a custom class-based TF-IDF to create dense clusters allowing for easily interpretable topics whilst keeping important words in the topic descriptions.

<br>

<img src="https://raw.githubusercontent.com/MaartenGr/BERTopic/master/images/logo.png" width="40%">

# Starta en GPU session


- Navigera till Edit -> Notebook Settings Navigate to Edit→Notebook Settings
- Välj GPU från Hardware Accelerator drop-down

[Reference](https://colab.research.google.com/notebooks/gpu.ipynb)

# **Installera BERTopic**

Först installeras BERTTopic från PyPi

In [1]:
%%capture
!pip install bertopic

## Sätt upp åtkomst till datafil

Eftersom filen är så stor så har jag sparat den på min gDrive. För att kunna excekvera notebooken och analysera data behöver ni skapa någon typ av lagringsplats där ni kan spara filen och läsa från den. Det kommer inte fungera att göra det från min gDrive.

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


## Starta om Notebook
När BERTTopic är installerad så behöver vissa andra paket också uppdateras, detta kräver en omstart.

Från menyn:

Körning → Starta om körning

## DATA

Vi använder en kurerad datamängd från den ursprungliga datamängd som tillhandahållits av Arbetsmiljöverket. Dock verkar något fortfarande ligga i filen som jag inte ser och därför fick jag göra en workaround för att få in texten från endast en kolumn.

In [1]:
import pandas as pd
temp = pd.read_csv('/content/drive/MyDrive/AI_AV/Webbstat_utdrag_AI_en_kolumn.csv', nrows=100000, names = [f'col{i+1}' for i in range(2)])
skador=temp['col1']
skador

0        Sociala och organisatoriska orsaker
1             Ergonomiska belastningsorsaker
2             Ergonomiska belastningsorsaker
3             Ergonomiska belastningsorsaker
4             Ergonomiska belastningsorsaker
                        ...                 
99995                                 Saknas
99996                                 Saknas
99997                                 Saknas
99998                                 Saknas
99999                                 Saknas
Name: col1, Length: 100000, dtype: object

Eftersom sentence transformers förväntar sig text i form av en mening i taget i en lista skapas en lista som består av alla texter.

In [3]:
skador.to_list

<bound method IndexOpsMixin.tolist of 0        Sociala och organisatoriska orsaker
1             Ergonomiska belastningsorsaker
2             Ergonomiska belastningsorsaker
3             Ergonomiska belastningsorsaker
4             Ergonomiska belastningsorsaker
                        ...                 
99995                                 Saknas
99996                                 Saknas
99997                                 Saknas
99998                                 Saknas
99999                                 Saknas
Name: col1, Length: 100000, dtype: object>

# **Temamodellering**

Nedan följer ett exempel på hur en temamodell skapas utifrån den inlästa datamängden som alltså består av en lista med 100 000 meningar som lagts en och en i en lista.




## Träning
**OBS:** *Temamodellering är en oövervakad inlärningsmodell, den tränas alltså inte på märkt data utan förväntas hitta mönster på egen hand.*

Vi börjar med att instansiera den sentence transformer vi vill använda och sparar den i en variabel. Därefter instansierar vi själva BERTTopic. Vi anger att vår sentence transformer från KB-lab ska användas eftersom det är svensk text vi ska analysera.  För att använda en sentence transformer som kan hantera 50+ språk (men förmodligen sämre) kan istället en parameter för språk multilingual `language="multilingual"`

Det är också möjligt att sätta andra parametrar, exempelvis sannolikhetsparametrar för olika teman. Vissa av dessa gör dock algoritmerna långsammare så har man mycket data är det klokt att inte slå på dessa parametrar.


In [4]:
#hämtar KB-lab sentence transformer och instansierar den i variabeln model_SWE
from sentence_transformers import SentenceTransformer
model_SWE = SentenceTransformer('KBLab/sentence-bert-swedish-cased')




In [5]:
#importera BERTTopic och instansiera  modellen i variablen 'topic_model'.
from bertopic import BERTopic

topic_model = BERTopic(embedding_model= model_SWE, calculate_probabilities=False)
#träna modellen med den lista med meningar som finns i variablen 'skador' och spara resultatet i variablen 'topics'
topics = topic_model.fit_transform(skador)

# **Embedding Models**
Parametern  `embedding_model` tar en sträng som input som pekar på en viss sentence-transformers modell, en SentenceTransformer, eller en Flair DocumentEmbedding model. Det är därmed mycket enkelt att testa olika embeddingmodeller. I grundutförandet av BERTTopic används SBERT. Här har KB:s modell 'KBLab/sentence-bert-swedish-cased' använts vilket är en sentence-transformer tränad på svensk text som publicerats på Hugginface.

## Extrahera Teman (topics)
När vi tränat (fitted) vår modell så kan vi titta på resultaten. Ofta vill vi titta på de teman som är vanligast eftersom de bäst representerar de texter vi analyserat. Nedan visas de fem teman som är vanligast

In [6]:
freq = topic_model.get_topic_info(); freq.head(5)

,Topic,Count,Name,Representation,Representative_Docs
0,-1,26590,-1_ergonomiska_belastningsorsaker_saknas_,"[ergonomiska, belastningsorsaker, saknas, , , ...","[Ergonomiska belastningsorsaker, Ergonomiska b..."
1,0,6846,0_sociala_organisatoriska_och_orsaker,"[sociala, organisatoriska, och, orsaker, , , ,...","[Sociala och organisatoriska orsaker, Sociala ..."
2,1,3760,1_kemiska_eller_biologiska_orsaker,"[kemiska, eller, biologiska, orsaker, , , , , , ]","[Kemiska eller biologiska orsaker, Kemiska ell..."
3,2,2206,2_fysikaliska_orsaker_vibrationer_,"[fysikaliska, orsaker, vibrationer, , , , , , , ]","[Fysikaliska orsaker, Fysikaliska orsaker, Fys..."
4,3,859,3_buller___,"[buller, , , , , , , , , ]","[Buller, Buller, Buller]"


-1 refererar till alla avvikande värden (outliers) och ska oftast ignoreras.

In [7]:
topic_model.get_topic(0)  # Select the most frequent topic

[('sociala', 0.0013475158270583939),
 ('organisatoriska', 0.0013475158270583939),
 ('och', 0.0013475158270583939),
 ('orsaker', 0.0007209950825272812),
 ('', 1e-05),
 ('', 1e-05),
 ('', 1e-05),
 ('', 1e-05),
 ('', 1e-05),
 ('', 1e-05)]

**VIKTIGT**: BERTopic är en stokastisk modell vilket innebär att teman kan skilja sig mellan olika körningar. Det beror främst på att UMAP är stokastisk.

## Attribut

Det finns ett antal attribut som går att nå efter att din BERTTopic modell har tränats (dessa har inte översatts eftersom det är attribut):


| Attribute | Description |
|------------------------|---------------------------------------------------------------------------------------------|
| topics_               | The topics that are generated for each document after training or updating the topic model. |
| probabilities_ | The probabilities that are generated for each document if HDBSCAN is used. |
| topic_sizes_           | The size of each topic                                                                      |
| topic_mapper_          | A class for tracking topics and their mappings anytime they are merged/reduced.             |
| topic_representations_ | The top *n* terms per topic and their respective c-TF-IDF values.                             |
| c_tf_idf_              | The topic-term matrix as calculated through c-TF-IDF.                                       |
| topic_labels_          | The default labels for each topic.                                                          |
| custom_labels_         | Custom labels for each topic as generated through `.set_topic_labels`.                                                               |
| topic_embeddings_      | The embeddings for each topic if `embedding_model` was used.                                                              |
| representative_docs_   | The representative documents for each topic if HDBSCAN is used.                                                |

Ett exempel nedan, hämtar förutspådda teman för de första 10 dokumenten (raderna).

In [8]:
topic_model.topics_[:10]

[0, -1, 1693, -1, 2355, -1, -1, 2143, -1, 1]

# **Visualiseringar**

**OBS** *Det är viktigt att installera BERTTopic och sedan starta om den virtuella maskinen eftersom visualiseringar inte kommer fungera annars på grund av att BERTTopic behöver en äldre version av ett visualiseringsbibliotek än vad som är default i Colaboratory. Startas maskinen inte om så. fungerar fortfarande analysen men flera av visualiseringarna syns inte.*

Det finns flera olika möjligheter för att visualisera resultatet. Främst handlar det om att visualisera teman, sannolikheter för olika teman och teman över tid.

Eftersom temagenerering är att betrakta som subjektivt kan visualiseringar hjälpa till att förstå de teman som skapats.

## Visualisera Topics
Efter att vi tränat `BERTopic` modellen, kan vi iterativt. gå igenom ungefär de första 100 teman som skapats för att få en förståelse för de teman som skapats. Det tar dock rätt mycket tid och det blir svårt att få en god överblick. Därför kan. man istället visualisera de teman som skapats. Visualiseringen är inspirerad av [LDAvis](https://github.com/cpsievert/LDAvis):

In [ ]:
topic_model.visualize_topics()

## Visualisera Topic Probabilities OBS inte möjligt att utföra med nuvarande modell

Vi har inte bett modellen att. beräkna `probabilities`  men hade vi gjort det kan vi använda resultatet för att undersöka hur säker modellen är på att ett visst tema finns i ett visst dokument. Då hade vi dock behövt beräkna probability när modellen skapas, vilket gör den betydligt långsammare, vilket inte var möjligt med den miljö testerna utförts i. För att beräkna probablilty ska attributet 'calculate_probabilities' sättas till True istället för False när modellen instansieras.

För att skapa en sådan visualisering (givet att attributet inkluderats i träningen av modellen) används följande kod:

In [ ]:
topic_model.visualize_distribution(probs[200], min_probability=0.015)

## Visualisera hierarkier i teman

De teman som skapats kan reduceras hierarkiskt. För att förstå den potentiella hierarkiska strukturen kan kluster skapas som visar hur olika teman relaterar till varandra. Det kan hjälpa när man vill minska antalet teman genom att slå samman dem.

In [16]:
topic_model.visualize_hierarchy(top_n_topics=10)

## Visualisera ord

Det går också att visuellt visa de ord som valts ut av modellen för att skapa ett tema. Exempelvis kan vi visualisera några teman genom att skapa bargrafer av de c-TF-IDF poäng som genererats för varje temas representation. Detta kan generera ytterligare förståelse genom att man kan jämföra poängen mellan och inom olika teman.

In [17]:
topic_model.visualize_barchart(top_n_topics=10)

## Visualisera likhet mellan teman
När vi genererat embeddings med både c-TF-IDF and sentence_transformers, kan vi skapa en matris som visar likheten baserat på en cosinus formel. Resultatet blir då en matris som indikerar hur lika olika teman är varandra.

In [21]:
topic_model.visualize_heatmap(n_clusters=10, width=1000, height=1000)

## Visualisering av Term Score Decline
Teman representeras av ett antal ord där det ord som bäst representerar temat presenteras först. Varje ord har en poäng som kommer från den embeddning som gjorts av c-TF-IDF algoritmen. Ju högre poäng, desto bättre representerar ett visst ord det tema det ingår i. Eftersom orden som representerar temat är rangordnade efter sin poäng så kommer varje ord i ordningen har mindre koppling till sitt tema. Vid någon punkt tillför inte längre tillagt ord något till sitt tema och är därmed inte en god representant för temat.

FÖr att visualisera detta kan poängen i enlighet med c-TF-IDF för varje tema plottas.

Med andra ord så plottas positionerna för varje ord i ett tema på x-axeln där det ord som har bäst poäng får rankingen 1 medan sämre rankade ord hamnar lägre, och lägre på x-axeln. Vad vi då kan se är hur poängen hela tiden minskar när fler ord läggs till i ett tema. Det kan vara till hjälp för att göra inställningar för hur många ord som optimalt ska ingå i ett tema för att temat ska vara enhetligt och förståeligt. En metod för att välja antal ord är armbågsmetoden som beskrivs såhär på wikipedia [elbow method](https://en.wikipedia.org/wiki/Elbow_method_(clustering))


In [ ]:
topic_model.visualize_term_rank()

# **Tema representation**
När en modell med teman skapats kan dessa teman uppdateras för att bättre fånga en text. Därmed finjusteras modellen efter specifika önskemål.



## Att uppdatera teman
Exempel på anledningar att uppdatera teman kan vara att vissa teman inte är relevanta, kanske beroende på att de bygger på triviala ord (stoppord) som inte borde ingått i analysen. Eller av andra anledningar. FÖr att uppdatera modellen kan funktionen `update_topics` användas. Det som då går att ändra är parametrarna i `c-TF-IDF` som bestämmer vilka ord som utgör ett specifikt tema:


In [ ]:
topic_model.update_topics(sampled_corpus, n_gram_range=(1, 2))

In [ ]:
topic_model.get_topic(0)   # We select topic that we viewed before

[('call', 0.009435313911423536),
 ('phone', 0.008125493949572665),
 ('me', 0.0075528166790997655),
 ('calls', 0.007511304279367785),
 ('number', 0.007291605874965206),
 ('calling', 0.007076931103879009),
 ('she', 0.007069637205779178),
 ('they', 0.006978940636029365),
 ('he', 0.0066316271499559775),
 ('debt', 0.006513555143206685)]

## Minska antalet teman Topic Reduction
Det är också möjligt att minska antalet teman efter att modellen tränats. Fördelen med detta är att du därmed kan välja antal teman efter att du tittat på vilka teman som skapats, istället för att gissa det mest lämpliga antalet baserat på fritexten som ingår i analysen:





In [ ]:
topic_model.reduce_topics(sampled_corpus, nr_topics=60)

2023-08-27 19:32:27,434 - BERTopic - Reduced number of topics from 161 to 60


In [ ]:
# Access the newly updated topics with:
print(topic_model.topics_)

[0, 32, -1, -1, -1, -1, -1, 0, 0, -1, -1, -1, -1, -1, -1, 14, -1, -1, -1, 4, 6, -1, -1, 4, 0, -1, -1, -1, -1, 20, -1, 48, 5, 0, 25, 11, 24, -1, 4, -1, -1, 23, 51, -1, 0, -1, -1, 7, 1, 5, -1, -1, 48, 1, -1, 4, -1, -1, -1, -1, 0, -1, -1, -1, -1, -1, 0, -1, -1, -1, -1, 19, 35, -1, -1, -1, 0, 37, -1, 0, 54, 6, 58, 51, 30, -1, -1, 19, -1, -1, 0, 2, 45, 32, -1, -1, 21, -1, -1, -1, -1, -1, -1, 3, 2, 0, -1, -1, -1, -1, 11, -1, -1, 15, 14, -1, -1, -1, 0, 5, 5, -1, -1, -1, 9, -1, -1, 2, -1, -1, -1, -1, -1, 0, 55, 2, 21, 1, -1, -1, -1, 11, 31, -1, -1, -1, 11, 10, 0, 7, 4, 4, 55, -1, -1, -1, -1, 3, 8, 16, -1, 2, -1, -1, -1, -1, -1, -1, -1, 25, 24, -1, 28, -1, -1, -1, 0, -1, 4, 0, 6, 0, -1, -1, -1, 2, -1, 31, -1, -1, 5, -1, 2, 13, -1, -1, 14, -1, -1, 10, -1, -1, -1, -1, -1, -1, 18, -1, 53, 13, 4, 44, -1, 5, 4, -1, -1, -1, -1, 2, 0, 34, -1, 6, 1, -1, 1, 4, -1, 0, -1, -1, -1, 40, 0, -1, -1, 0, 0, -1, -1, -1, 25, 39, -1, 7, 2, 6, -1, 29, -1, -1, 40, -1, 23, -1, -1, -1, -1, -1, -1, -1, -1, -1, 32, 21, 

# **Söka efter teman**
Efter att modellen tränats kan funktionen `find_topics` användas för att söka efter teman som liknar ett specifikt sökord. I koden nedan söker funktionen efter teman som liknar söktermen "psykisk" och presenterar de teman som mest liknar ordet:

In [ ]:
similar_topics, similarity = topic_model.find_topics("psykisk", top_n=5); similar_topics

[71, 45, 77, 9, 56]

In [ ]:
topic_model.get_topic(71)

[('car', 0.03740731827314482),
 ('the car', 0.027790363401304377),
 ('dealer', 0.013837911908704722),
 ('the dealer', 0.009515109324321468),
 ('owner', 0.008430722097917726),
 ('previous owner', 0.008157988442865012),
 ('cars', 0.005827046491488879),
 ('the odometer', 0.00514870077683653),
 ('bought car', 0.004667512506960727),
 ('car with', 0.004498685875558186)]

# **Model serialization**
Modellen med dess interna inställningar kan sparas enkelt. Däremot sparas inte de data som ingår i analysen och inte heller de inbäddningar som skapas av BERT och c-TF_IDF. UMAP och HDBSCAN sparas dock.

In [ ]:
# Save model
topic_model.save("my_model")

In [ ]:
# Load model
my_model = BERTopic.load("my_model")

## Sentence-Transformers
Sokm nämts går det att välja vilken sentence-transformers modell som helst. Det görs genom att ange namnet på modellen i argumentet embedding_model.



In [ ]:
#Exempel på BERTTopic med en annan embedding som klarar 50+ språk
topic_model = BERTopic(embedding_model="xlm-r-bert-base-nli-stsb-mean-tokens")

[Här](https://www.sbert.net/docs/pretrained_models.html) finns en lista med supporterade sentence transformers modeller.  
